In [1]:
# Strong scaling test 1 (Also used as Weak scaling test 1)
# 3 nodes
# 9 GB of data
from pyspark.sql import SparkSession, SQLContext 
import json
import timeit
import pyspark.sql.functions as f 

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.225:7077") \
        .appName("Strong_scaling_3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.cores.max",3)\
        .getOrCreate()

In [2]:
%%time
start_time_0 = timeit.default_timer()

df = spark_session.read.json('hdfs://192.168.2.225:9000/reddit/RC_2012-02')

elapsed_0 = timeit.default_timer() - start_time_0

print("Time elapsed: ", elapsed_0, "s")

Time elapsed:  145.97712686506566 s
CPU times: user 48.4 ms, sys: 984 µs, total: 49.3 ms
Wall time: 2min 25s


In [3]:
df.count()

16015695

In [4]:
initial_start_time = timeit.default_timer()


if 'author_cakeday' in df.columns:
        current_df = df.drop('author_cakeday')
        
df = df.drop('author_flair_css_class','author_flair_text','can_gild','distinguished','edited','id','is_submitter','link_id','parent_id','permalink','retrieved_on','stickied','subreddit_id')

df = df.filter((df.body != '[deleted]'))


elapsed = timeit.default_timer() - initial_start_time



print("Time elapsed: ", elapsed, "s")

Time elapsed:  0.23167094099335372 s


In [5]:
%%time
start_time_1 = timeit.default_timer()

df.groupBy('subreddit').count().sort("count", ascending = False).show(10)

elapsed_1 = timeit.default_timer() - start_time_1

print("Time elapsed: ", elapsed_1, "s")

+-------------------+-------+
|          subreddit|  count|
+-------------------+-------+
|          AskReddit|1974313|
|              funny| 767566|
|               pics| 601277|
|            atheism| 482672|
|             gaming| 477368|
|                WTF| 391188|
|           politics| 363712|
|              trees| 296983|
|fffffffuuuuuuuuuuuu| 294680|
|               IAmA| 267945|
+-------------------+-------+
only showing top 10 rows

Time elapsed:  87.39740490203258 s
CPU times: user 20.3 ms, sys: 14.9 ms, total: 35.2 ms
Wall time: 1min 27s


In [6]:
%%time
start_time_2 = timeit.default_timer()

df = df.withColumn('wordCount', f.size(f.split(f.col('body'), ' ')))

df.groupBy('wordCount').count().sort("count", ascending = False).show(10)   

elapsed_2 = timeit.default_timer() - start_time_2


print("Time elapsed: ", elapsed_2, "s")

+---------+------+
|wordCount| count|
+---------+------+
|        5|578020|
|        6|576405|
|        4|557292|
|        7|554545|
|        8|527944|
|        1|509309|
|        3|508649|
|        9|495282|
|       10|465635|
|        2|454971|
+---------+------+
only showing top 10 rows

Time elapsed:  118.58791058906354 s
CPU times: user 33.4 ms, sys: 12.4 ms, total: 45.8 ms
Wall time: 1min 58s


In [7]:
%%time
start_time_3 = timeit.default_timer()

df_stats = df.select(f.mean(f.col('wordCount')).alias('mean')).collect()  #

mean = df_stats[0]['mean']

print("Average wordcount in comment: ", mean, " words.")


elapsed_3 = timeit.default_timer() - start_time_3
print("Time elapsed: ", elapsed_3, "s")

Average wordcount in comment:  32.56660286612964  words.
Time elapsed:  111.29889625695068 s
CPU times: user 34.3 ms, sys: 18 ms, total: 52.4 ms
Wall time: 1min 51s


In [8]:
total_elapsed = timeit.default_timer() - initial_start_time

print("Total time elapsed: ", total_elapsed, "s")

Total time elapsed:  317.6207772109192 s


In [9]:
spark_session.stop()